In [2]:
# IMPORTING LIBRARIES
import random
import numpy as np

In [3]:
# CONFIG SETUP
suits = ['C','D','H','S']
suits = ["♠","♣","♥","♦" ]
ranks  = {  "J":3,
            "9":2,
            "A":1,"10":1,
            "K":0,"Q":0
}
delim = "|"
num_players=  4
starting_player = 1


In [4]:
def cards_distributor(num_players = 4):
    
    cards = [[i+delim+k for k in suits] for i in ranks.keys()]
    cards = [i for j in cards for i in j]
    random.shuffle(cards)
    players_deck = {k+1:list(hand) for k, hand in enumerate(np.array_split(cards,num_players))}
    return players_deck

In [5]:
def deck_power(deck):
    power = np.sum([ranks[i.split(delim)[0]] for i in deck])
    return power

def trump_power(deck):
    trumps = {k:0 for k in suits}
    for card in deck: 
        trumps[card.split(delim)[-1]]=trumps[card.split(delim)[-1]] + 1 + ranks[card.split(delim)[0]]
    return {k:v for k,v  in sorted(trumps.items(), key = lambda x: -x[1])}

def play_order(first_player,num_players):
    
    all_players = list(range(1,num_players+1))
    player_index = all_players.index(first_player)
    return all_players[player_index:] + all_players[:player_index]


In [124]:
print("Starting the Game of 28!")


print("\nDistributing cards")
player_decks = cards_distributor(num_players)
players_dict = {}
for player, deck in player_decks.items():
    players_dict[player] = {} 
    sorted_deck = sorted(deck, key = lambda x: -ranks[x.split(delim)[0]])
    players_dict[player]['cards'] = sorted_deck
    players_dict[player]['deck_power'] = deck_power(deck)
    players_dict[player]['trump_power'] = trump_power(deck)

    if player%2==0: players_dict[player]['team'] = 'B'
    else: players_dict[player]['team'] = 'A'

    print(f" \n Player {player}: {', '.join(sorted_deck)} \t Power: {deck_power(deck)}")
    print(f"\t{trump_power(deck)}")

    initial_players_dict = players_dict.copy()

Starting the Game of 28!

Distributing cards
 
 Player 1: J|♦, J|♠, 10|♦, A|♥, A|♣, 10|♣ 	 Power: 10
	{'♦': 6, '♠': 4, '♣': 4, '♥': 2}
 
 Player 2: 9|♠, 9|♦, 10|♠, A|♠, A|♦, Q|♣ 	 Power: 7
	{'♠': 7, '♦': 5, '♣': 1, '♥': 0}
 
 Player 3: 9|♣, 10|♥, K|♦, K|♠, K|♣, Q|♠ 	 Power: 3
	{'♣': 4, '♠': 2, '♥': 2, '♦': 1}
 
 Player 4: J|♣, J|♥, 9|♥, K|♥, Q|♦, Q|♥ 	 Power: 8
	{'♥': 9, '♣': 4, '♦': 1, '♠': 0}


In [125]:
no_of_rounds = int((len(ranks)*len(suits))/num_players)
trump = random.choice(suits)

print(trump)

players_dict = dict(initial_players_dict)
def valid_play(round_cards, player_cards, trump = '' ):
    valid_suit =  [i for i in player_cards if i.split(delim)[-1]==round_cards[0].split(delim)[-1]]
    if len(valid_suit)>0: return valid_suit
    trumps = [i for i in player_cards if i.split(delim)[-1]==trump]

    if len(trumps) == 0:
        return player_cards
    return trumps


def next_round_initiator(round_cards, round_order, trump):
    scores = [ranks[card.split(delim)[0]]   for card in round_cards if card.split(delim)[-1] == round_cards[0].split(delim)[-1] ]
    score_map =  dict(zip(round_order, scores))
    score_map =  {k: v for k, v in sorted(score_map.items(), key=lambda item: -item[1])}

    vettu = [(player, card) for (player, card) in zip(round_order, round_cards) if card.split(delim)[-1]==trump]
    vettu = sorted(vettu, key = lambda x: -ranks[x[1].split(delim)[0]])
    if  vettu!=[]:
        return play_order(vettu[0][0], num_players)
    print("NO VETTU")
    # print(list(score_map))
    return play_order(list(score_map)[0], num_players)

def scoring(round_cards, round_order, players_dict):
    vettu = [(player, card) for (player, card) in zip(round_order, round_cards) if card.split(delim)[-1]==trump]
    vettu = sorted(vettu, key = lambda x: -ranks[x[1].split(delim)[0]])
    

    total_score = [ranks[card.split(delim)[0]] for card in round_cards]

    if len(vettu)>0:
        biggest_vettu = vettu[0]
        winner_player = biggest_vettu
    else:
        play_wise_scores = list(zip(round_order, total_score))
        play_wise_scores = sorted(play_wise_scores, key = lambda x: -x[1])
        winner_player = play_wise_scores[0]
        # return 

    print(f"Round Winner: {players_dict[winner_player[0]]['team']} - {sum(total_score)} points")
    return players_dict[winner_player[0]]['team'], sum(total_score)
    



first_player = starting_player
round_wise_cards = {}
scores_tally = {}
for round_no in range(1,no_of_rounds+1):
    print(f"\n ### ROUND {round_no} ###")
    round_wise_cards[round_no] = {'first_player':starting_player, 'play_order':play_order(first_player, num_players)}
    print(play_order(first_player,num_players))

    round_cards = []
    for play in round_wise_cards[round_no]['play_order']:
        if len(round_cards)==0:
            first_choice = players_dict[play]['cards'][random.randint(0,len(players_dict[play]['cards'])-1)]
            round_cards = [first_choice]
            players_dict[play]['cards'].remove(first_choice)
            # print(round_cards)

        else:
            eligible_play = valid_play(round_cards,players_dict[play]['cards'] , trump)
            # if eligible_play == []: eligible_play = 'TRUMP'
            round_cards.append(eligible_play[0])
        
        
    print(list(zip(round_wise_cards[round_no]['play_order'],round_cards)))
    # print(next_round_initiator(round_cards, play_order(first_player,num_players), trump))
    first_player = next_round_initiator(round_cards, play_order(first_player,num_players), trump)[0]

    team_name, round_score = scoring(round_cards, play_order(first_player,num_players), players_dict)
        



♦

 ### ROUND 1 ###
[1, 2, 3, 4]
[(1, 'A|♥'), (2, '9|♦'), (3, '10|♥'), (4, 'J|♥')]
Round Winner: A - 7 points

 ### ROUND 2 ###
[2, 3, 4, 1]
[(2, '9|♠'), (3, 'K|♠'), (4, 'Q|♦'), (1, 'J|♠')]
Round Winner: B - 5 points

 ### ROUND 3 ###
[4, 1, 2, 3]
[(4, 'K|♥'), (1, 'J|♦'), (2, '9|♦'), (3, '10|♥')]
Round Winner: B - 6 points

 ### ROUND 4 ###
[1, 2, 3, 4]
[(1, 'A|♣'), (2, 'Q|♣'), (3, '9|♣'), (4, 'J|♣')]
NO VETTU
Round Winner: A - 6 points

 ### ROUND 5 ###
[4, 1, 2, 3]
[(4, 'Q|♦'), (1, 'J|♦'), (2, '9|♦'), (3, 'K|♦')]
Round Winner: B - 5 points

 ### ROUND 6 ###
[1, 2, 3, 4]
[(1, '10|♣'), (2, 'Q|♣'), (3, '9|♣'), (4, 'J|♣')]
NO VETTU
Round Winner: A - 6 points
